---
# Parsing CSV And Excel Files

---

## CSV And Excel Files - Structured Data

---

In [1]:
import pandas as pandas
import os

In [2]:
os.makedirs("data/structured_files", exist_ok=True)

In [5]:
# Create sample data
data = {
    'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'Category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Electronics'],
    'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
    'Stock': [50, 200, 150, 75, 100],
    'Description': [
        'High-performance laptop with 16GB RAM and 512GB SSD',
        'Wireless optical mouse with ergonomic design',
        'Mechanical keyboard with RGB backlighting',
        '27-inch 4K monitor with HDR support',
        '1080p webcam with noise cancellation'
    ]
}
data

{'Product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
 'Category': ['Electronics',
  'Accessories',
  'Accessories',
  'Electronics',
  'Electronics'],
 'Price': [999.99, 29.99, 79.99, 299.99, 89.99],
 'Stock': [50, 200, 150, 75, 100],
 'Description': ['High-performance laptop with 16GB RAM and 512GB SSD',
  'Wireless optical mouse with ergonomic design',
  'Mechanical keyboard with RGB backlighting',
  '27-inch 4K monitor with HDR support',
  '1080p webcam with noise cancellation']}